This script checks if `numpy` and `pygame` are installed. If not, it installs them. Afterward, it imports essential libraries (`numpy`, `pygame`, `sys`, `random`, `math`, `time`). Run the script to ensure all dependencies are available for use in your Python environment.

In [ ]:
# install packages
try:
    __import__('numpy')
    print('numpy is already installed')
except ImportError:
    print("package not found installing")
    %pip install numpy

try:
    __import__('pygame')
    print('numpy is already installed')
except ImportError:
    print("package not found installing")
    %pip install pygame

In [5]:
import numpy as np
import pygame
import sys
import random
import math
import time
import threading

# Game Constants
ROW_COUNT = 6
COLUMN_COUNT = 7
PLAYER = 1  # Minimax-controlled side (Red)
AI = 2      # MCTS-controlled side (Yellow)
EMPTY = 0
WINDOW_LENGTH = 4
running = True
ai_move_result = None
minimax_total_time = 0
mcts_total_time = 0
minimax_move_count = 0
mcts_move_count = 0

# Colors (RGB)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)

Run the code below to launch the connect4 GUI. Here we are evuating the Minimax with Alpha-Beta pruning and the MCTS algorithms, each time the cell block is ran, the algorithms will play 7 games each as the starting player and will iterate through the 7 starting positions. In total , 14 games will be played and the average time take per move by each algorithm will be printed as well as which algorithm won or if there is a tie.

In [8]:
# Pygame Setup
SQUARESIZE = 100
width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT + 2) * SQUARESIZE
size = (width, height)
RADIUS = int(SQUARESIZE / 2 - 5)
pygame.init()
FONT = pygame.font.SysFont("monospace", 50)
screen = pygame.display.set_mode(size)

def create_board():
    return np.zeros((ROW_COUNT, COLUMN_COUNT))

def is_valid_location(board, col):
    return board[ROW_COUNT - 1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def winning_move(board, piece):
    # Check horizontal
    for r in range(ROW_COUNT):
        for c in range(COLUMN_COUNT - 3):
            if all(board[r][c + i] == piece for i in range(WINDOW_LENGTH)):
                return True
    # Check vertical
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT - 3):
            if all(board[r + i][c] == piece for i in range(WINDOW_LENGTH)):
                return True
    # Check positively sloped diagonals
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            if all(board[r + i][c + i] == piece for i in range(WINDOW_LENGTH)):
                return True
    # Check negatively sloped diagonals
    for r in range(3, ROW_COUNT):
        for c in range(COLUMN_COUNT - 3):
            if all(board[r - i][c + i] == piece for i in range(WINDOW_LENGTH)):
                return True
    return False

def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER if piece == AI else AI
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2
    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4
    return score

def score_position(board, piece):
    score = 0
    # Score center column
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT // 2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score Horizontal
    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(COLUMN_COUNT - 3):
            window = row_array[c:c + WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Score Vertical
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(ROW_COUNT - 3):
            window = col_array[r:r + WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Score positively sloped diagonal
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + i][c + i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    # Score negatively sloped diagonal
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + 3 - i][c + i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    return score

# ------------------- MCTS Components -------------------
class Node:
    def __init__(self, board, parent=None, move=None):
        self.board = board
        self.parent = parent
        self.move = move
        self.children = []
        self.wins = 0
        self.visits = 0
        self.untried_moves = self.get_legal_moves()

    def get_legal_moves(self):
        return [col for col in range(COLUMN_COUNT) if is_valid_location(self.board, col)]

    def select_child(self):
        exploration_weight = 1.414  # sqrt(2)
        best_score = -float('inf')
        best_child = None
        for child in self.children:
            exploit = child.wins / child.visits
            explore = exploration_weight * math.sqrt(math.log(self.visits) / child.visits)
            score = exploit + explore
            if score > best_score:
                best_score = score
                best_child = child
        return best_child

    def add_child(self, move, board):
        child = Node(board, parent=self, move=move)
        self.untried_moves.remove(move)
        self.children.append(child)
        return child

    def update(self, result):
        self.visits += 1
        self.wins += result

def mcts(root, simulations=1000, piece=AI):
    """Monte Carlo Tree Search for the given piece."""
    for _ in range(simulations):
        node = root
        # 1. Selection
        while node.untried_moves == [] and node.children != []:
            node = node.select_child()
        # 2. Expansion
        if node.untried_moves:
            move = random.choice(node.untried_moves)
            row = get_next_open_row(node.board, move)
            board_copy = node.board.copy()
            drop_piece(board_copy, row, move, piece)
            node = node.add_child(move, board_copy)
        # 3. Simulation
        board_copy = node.board.copy()
        current_turn = piece
        while not winning_move(board_copy, PLAYER) and not winning_move(board_copy, AI):
            valid_moves = [col for col in range(COLUMN_COUNT) if is_valid_location(board_copy, col)]
            if not valid_moves:
                break
            move = random.choice(valid_moves)
            row = get_next_open_row(board_copy, move)
            drop_piece(board_copy, row, move, current_turn)
            current_turn = PLAYER if current_turn == AI else AI
        # 4. Backpropagation
        while node is not None:
            result = 1 if winning_move(board_copy, piece) else 0
            node.update(result)
            node = node.parent
    if root.children:
        best_move = max(root.children, key=lambda x: x.visits).move
    else:
        best_move = None
    return best_move

# ------------------- Minimax Algorithm -------------------
def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = [col for col in range(COLUMN_COUNT) if is_valid_location(board, col)]
    random.shuffle(valid_locations)
    is_terminal = winning_move(board, PLAYER) or winning_move(board, AI) or len(valid_locations) == 0
    if depth == 0 or is_terminal:
        if winning_move(board, AI):
            return (None, 100000000000000)
        elif winning_move(board, PLAYER):
            return (None, -10000000000000)
        else:
            current_piece = AI if maximizingPlayer else PLAYER
            return (None, score_position(board, current_piece))
    if maximizingPlayer:
        value = -float('inf')
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI)
            new_score = minimax(b_copy, depth - 1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value
    else:
        value = float('inf')
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER)
            new_score = minimax(b_copy, depth - 1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

# ------------------- Drawing the Board and Legend -------------------
def draw_board(board, title, game_over):
    screen.fill(WHITE)
    # Draw title and legend in the top area (first 2 rows are reserved for info)
    title_text = FONT.render(title, True, BLACK)
    screen.blit(title_text, (20, 10))
    legend_minimax = FONT.render("Red: Minimax", True, RED)
    legend_mcts = FONT.render("Yellow: MCTS", True, YELLOW)
    screen.blit(legend_minimax, (20, 60))
    screen.blit(legend_mcts, (20, 110))
    
    # Draw grid and holes starting from row 2
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, BLUE, (c * SQUARESIZE, (r + 2) * SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BLACK, (int(c * SQUARESIZE + SQUARESIZE / 2),
                                                 int((r + 2) * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
    # Draw pieces
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == PLAYER:
                pygame.draw.circle(screen, RED, (int(c * SQUARESIZE + SQUARESIZE / 2),
                                                   height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[r][c] == AI:
                pygame.draw.circle(screen, YELLOW, (int(c * SQUARESIZE + SQUARESIZE / 2),
                                                      height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
    # If game over, display winner message
    if game_over:
        if winning_move(board, PLAYER):
            win_text = FONT.render("Minimax (PLAYER) wins!", True, RED)
        elif winning_move(board, AI):
            win_text = FONT.render("MCTS (AI) wins!", True, YELLOW)
        else:
            win_text = FONT.render("It's a tie!", True, BLUE)
        screen.blit(win_text, (20, SQUARESIZE + 20))
    pygame.display.update()

def main():
    global running, ai_move_result, minimax_total_time, mcts_total_time
    global minimax_move_count, mcts_move_count
    pygame.init()
    screen = pygame.display.set_mode((COLUMN_COUNT * 100, (ROW_COUNT + 2) * 100))
    FONT = pygame.font.SysFont("monospace", 50)
    
    minimax_wins = 0
    mcts_wins = 0
    ties = 0
    game_num = 0

    def run_ai_thread(target_func, *args):
        global ai_move_result
        ai_move_result = target_func(*args)

    def check_events():
        global running
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    # First 7 games: Minimax starts
    for start_col in range(COLUMN_COUNT):
        if not running:
            break
        game_num += 1
        board = create_board()
        
        # Make starting move for Minimax
        if is_valid_location(board, start_col):
            row = get_next_open_row(board, start_col)
            drop_piece(board, row, start_col, PLAYER)
        
        turn = AI
        game_over = False
        
        while not game_over and running:
            check_events()
            draw_board(board, f"Game {game_num} (Minimax starts)", game_over)
            
            if turn == AI and not game_over:
                thread = threading.Thread(target=run_ai_thread, args=(mcts, Node(board), 1000, AI))
                thread.start()
                while thread.is_alive() and running:
                    check_events()
                    pygame.time.wait(100)
                
                if not running:
                    break
                
                col = ai_move_result
                
                # Validate move
                if col is not None and is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, AI)
                    mcts_move_count += 1
                    if winning_move(board, AI):
                        game_over = True
                        mcts_wins += 1
                    turn = PLAYER
                else:
                    valid_moves = [c for c in range(COLUMN_COUNT) if is_valid_location(board, c)]
                    col = random.choice(valid_moves) if valid_moves else None
                    if col is not None:
                        row = get_next_open_row(board, col)
                        drop_piece(board, row, col, AI)
                        mcts_move_count += 1
                        if winning_move(board, AI):
                            game_over = True
                            mcts_wins += 1
                        turn = PLAYER
            
            elif turn == PLAYER and not game_over:
                start_time = time.perf_counter()
                col, _ = minimax(board, 4, -math.inf, math.inf, False)
                elapsed_time = time.perf_counter() - start_time
                minimax_total_time += elapsed_time
                minimax_move_count += 1
                
                if col is not None and is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, PLAYER)
                    if winning_move(board, PLAYER):
                        game_over = True
                        minimax_wins += 1
                    turn = AI
                else:
                    valid_moves = [c for c in range(COLUMN_COUNT) if is_valid_location(board, c)]
                    col = random.choice(valid_moves) if valid_moves else None
                    if col is not None:
                        row = get_next_open_row(board, col)
                        drop_piece(board, row, col, PLAYER)
                        if winning_move(board, PLAYER):
                            game_over = True
                            minimax_wins += 1
                        turn = AI
            
            if len([col for col in range(COLUMN_COUNT) if is_valid_location(board, col)]) == 0:
                game_over = True
                ties += 1
            
            draw_board(board, f"Game {game_num} (Minimax starts)", game_over)
            pygame.time.wait(500)
        
        if running:
            print(f"Game {game_num} finished. Current score: Minimax {minimax_wins}, MCTS {mcts_wins}, Ties {ties}")

    # Next 7 games: MCTS starts
    for start_col in range(COLUMN_COUNT):
        if not running:
            break
        game_num += 1
        board = create_board()
        
        # Make starting move for MCTS
        if is_valid_location(board, start_col):
            row = get_next_open_row(board, start_col)
            drop_piece(board, row, start_col, AI)
        
        turn = PLAYER
        game_over = False
        
        while not game_over and running:
            check_events()
            draw_board(board, f"Game {game_num} (MCTS starts)", game_over)
            
            if turn == PLAYER and not game_over:
                thread = threading.Thread(target=run_ai_thread, args=(minimax, board, 4, -math.inf, math.inf, False))
                thread.start()
                while thread.is_alive() and running:
                    check_events()
                    pygame.time.wait(100)
                
                if not running:
                    break
                
                col, _ = ai_move_result
                
                # Validate move
                if col is not None and is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, PLAYER)
                    minimax_move_count += 1
                    if winning_move(board, PLAYER):
                        game_over = True
                        minimax_wins += 1
                    turn = AI
                else:
                    valid_moves = [c for c in range(COLUMN_COUNT) if is_valid_location(board, c)]
                    col = random.choice(valid_moves) if valid_moves else None
                    if col is not None:
                        row = get_next_open_row(board, col)
                        drop_piece(board, row, col, PLAYER)
                        minimax_move_count += 1
                        if winning_move(board, PLAYER):
                            game_over = True
                            minimax_wins += 1
                        turn = AI
            
            elif turn == AI and not game_over:
                start_time = time.perf_counter()
                root = Node(board)
                col = mcts(root, 1000, AI)
                elapsed_time = time.perf_counter() - start_time
                mcts_total_time += elapsed_time
                mcts_move_count += 1
                
                if col is not None and is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, AI)
                    if winning_move(board, AI):
                        game_over = True
                        mcts_wins += 1
                    turn = PLAYER
                else:
                    valid_moves = [c for c in range(COLUMN_COUNT) if is_valid_location(board, c)]
                    col = random.choice(valid_moves) if valid_moves else None
                    if col is not None:
                        row = get_next_open_row(board, col)
                        drop_piece(board, row, col, AI)
                        mcts_move_count += 1
                        if winning_move(board, AI):
                            game_over = True
                            mcts_wins += 1
                        turn = PLAYER
            
            if len([col for col in range(COLUMN_COUNT) if is_valid_location(board, col)]) == 0:
                game_over = True
                ties += 1
            
            draw_board(board, f"Game {game_num} (MCTS starts)", game_over)
            pygame.time.wait(500)
        
        if running:
            print(f"Game {game_num} finished. Current score: Minimax {minimax_wins}, MCTS {mcts_wins}, Ties {ties}")

    # Calculate and print averages
    avg_minimax = minimax_total_time / minimax_move_count if minimax_move_count else 0
    avg_mcts = mcts_total_time / mcts_move_count if mcts_move_count else 0
    
    print("\nFinal Results:")
    print(f"Minimax Wins: {minimax_wins}")
    print(f"MCTS Wins: {mcts_wins}")
    print(f"Ties: {ties}")
    print(f"Average Minimax move time: {avg_minimax:.4f} seconds ({minimax_move_count} moves)")
    print(f"Average MCTS move time: {avg_mcts:.4f} seconds ({mcts_move_count} moves)")
    
    pygame.quit()
    return

if __name__ == "__main__":
    main()


Final Results:
Minimax Wins: 0
MCTS Wins: 0
Ties: 0
Average Minimax move time: 0.1124 seconds (76 moves)
Average MCTS move time: 0.0000 seconds (81 moves)
